In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlMoving
from python.fluorophores import FlMoveBleach
from python.simulators import SimSequencefile

In [ ]:
sim = SimSequencefile()

In [ ]:
fname = os.path.join(SCRIPT_DIR, "examples", "Tracking_2D.json")
fname2 = os.path.join(SCRIPT_DIR, "settings", "PSFvectorial2D.json") # use a PSF that is defined via a json file
sim.loadsequence(fname, fname2)
sim.makepatterns()

## Make diffusing, bleaching fluorophores

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(221)

sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]
sim.time = 0
sim.background = 0
fl = FlMoveBleach()
fl.photonbudget = 100000
updatetime = 0.01  # ms
D = 0.5  # um^2/s
fl.makediffusion(D, updatetime)
# diffusion coefficient, update time args.startpos,dim, numpoints, buondarybox
sim.fluorophores = fl

out = sim.runSequence(repetitions=1)
sim.plotpositions(out, xvalues="time")



In [ ]:
ax = fig.add_subplot(222)
ax.plot(out.loc.xnm, out.loc.ynm, color='k')
ax.plot(out.loc.xfl1,out.loc.yfl1, color='r')
ax.plot(out.loc.xgalvo+out.loc.xeod, out.loc.ygalvo+out.loc.yeod, color='g')
ax.legend(['estimated','fluorophore','galvo+EOD'])
ax.set_aspect('equal')
ax.set_xlabel("x (nm)")
ax.set_ylabel("y (nm)")
ax.set_title("diffusion")

## Make stepping fluorophore

In [ ]:
sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]
sim.time = 0
fl2 = FlMoveBleach()
fl2.photonbudget = 5000
fl2.brightness = 200
updatetime = 0.01  # us
stepsize = 16  # nm
dwelltime = 28  # ms
fl2.makesteps(stepsize, dwelltime, updatetime, angle=0, startpos=[50,0,0])

sim.fluorophores = fl2
sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]
sim.time = 0
out=sim.runSequence(repetitions=1)

In [ ]:
fig.add_subplot(223)
sim.plotpositions(out, xvalues="time")
ax.set_title("stepping")

In [ ]:
fig.tight_layout()

## Instabilities: vibrations

In [ ]:
fl3 = FlMoving(brightness=50000) # collect more photons
fl3.posmode = 'function'

frequencies = [0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10]  # kHz

amplitude = 5 # nm

stdx = np.zeros(len(frequencies))
stdxrel = np.zeros(len(frequencies))

for k in range(len(frequencies)):
    posfl = [0, 0, 0]
    fl3.posfunction = [lambda t: amplitude*np.sin(frequencies[k]*t) + posfl[0],
                       lambda t: 0*t+posfl[1],
                       lambda t: 0*t+posfl[2]]
    sim.fluorophores = fl3
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time = 0
    out = sim.runSequence(repetitions=10)
    filter = (out.loc.itr==max(out.loc.itr)) & (out.loc.vld==1)
    sr = sim.summarize_results(out, display=False, filter=filter)
    stdx[k] = sr.stdraw[0]
    stdxrel[k] = sr.stdraw[0]/sr.sCRB[0]


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(121)
ax.semilogx(frequencies,stdxrel)
ax.set_xlabel('frequncy (kHz)')
ax.set_ylabel('std(x)/sCRB(x)')
ax.set_title("vibrations: frequency")

In [ ]:
frequency=0.1
amplitudes = range(0,12,2)
stdxa = np.zeros(len(amplitudes))
stdxrela = np.zeros(len(amplitudes))
for k in range(len(amplitudes)):
    posfl = [0, 0, 0]
    fl3.posfunction = [lambda t: amplitudes[k]*np.sin(frequency*t)+posfl[0],
                       lambda t: 0*t + posfl[1],
                       lambda t: 0*t + posfl[2]]
    sim.fluorophores = fl3
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time = 0
    out=sim.runSequence(repetitions=10)
    filter = (out.loc.itr==max(out.loc.itr)) & (out.loc.vld==1)
    sr = sim.summarize_results(out, display=False, filter=filter)
    stdxa[k] = sr.stdraw[0]
    stdxrela[k] = sr.stdraw[0]/sr.sCRB[0]

In [ ]:
ax = fig.add_subplot(122)
ax.plot(amplitudes,stdxrela)
ax.set_xlabel('amplitude (nm)')
ax.set_ylabel('std(x)/sCRB(x)')
ax.set_title("vibrations: amplitude")

In [ ]:
fig.tight_layout()